## preprocessing using the library functions

I've poured all of the preprocessing steps so far into a `garbageman` library, which should handle all of the intermediate steps no problem.

In [1]:
import garbageman

In [2]:
import pandas as pd
import geopandas as gpd

buildings = gpd.read_file("../data/SF_Bldg_201610.gdb.zip").to_crs({"init": "epsg:4326"})
blocks = gpd.read_file("../data/sf-census-blocks.geojson")

In [11]:
%time blocks = garbageman.pipeline.drop_noncontiguous_blocks(blocks)

CPU times: user 979 ms, sys: 7.93 ms, total: 987 ms
Wall time: 989 ms


In [12]:
%time matches, _, _ = garbageman.pipeline.join_bldgs_blocks(buildings, blocks)

CPU times: user 2min 6s, sys: 2.79 s, total: 2min 9s
Wall time: 2min 13s


In [17]:
%time blockfaces = garbageman.pipeline.blockfaces_for_blocks(blocks)

CPU times: user 28.2 s, sys: 320 ms, total: 28.5 s
Wall time: 28.8 s


In [14]:
%time streets = gpd.read_file("../data/sf-street-centerlines.geojson")

CPU times: user 2.24 s, sys: 60.6 ms, total: 2.3 s
Wall time: 2.36 s


In [18]:
%time street_geospatial_index = garbageman.pipeline.build_streets_geospatial_index(streets)

CPU times: user 4.4 s, sys: 30.4 ms, total: 4.43 s
Wall time: 4.47 s


In [20]:
%time street_segments = garbageman.pipeline.merge_street_segments_blockfaces_blocks(blockfaces, streets, street_geospatial_index)

36182it [05:18, 113.72it/s]
36182it [00:55, 657.17it/s] 


CPU times: user 6min 18s, sys: 8.54 s, total: 6min 26s
Wall time: 6min 35s


In [47]:
!rm ../data/blockfaces.*

rm: ../data/blockfaces.*: No such file or directory


In [48]:
%time matches.to_file('../data/processed/buildings.geojson', driver='GeoJSON')

CPU times: user 7min 55s, sys: 5.04 s, total: 8min
Wall time: 8min 19s


In [53]:
%time blocks.to_file('../data/processed/blocks.geojson', driver='GeoJSON')

CPU times: user 6.01 s, sys: 301 ms, total: 6.31 s
Wall time: 6.34 s


In [59]:
!rm ../data/processed/blockfaces.geojson

In [57]:
blockfaces.head()

,geoid10,geoid10_n,geometry,simplified_geometry
0,060750213002002,060750213002002_1,"LINESTRING (-122.440616 37.750902, -122.440459...","LINESTRING (-122.440616 37.750902, -122.440459..."
1,060750213002002,060750213002002_2,"LINESTRING (-122.440459 37.749301, -122.441538...","LINESTRING (-122.440459 37.749301, -122.441538..."
2,060750213002002,060750213002002_3,"LINESTRING (-122.441538 37.74923100000001, -12...","LINESTRING (-122.441538 37.74923100000001, -12..."
3,060750213002002,060750213002002_4,"LINESTRING (-122.441687 37.75083600000001, -12...","LINESTRING (-122.441687 37.75083600000001, -12..."
0,060750213002000,060750213002000_1,"LINESTRING (-122.438397 37.750226, -122.43832 ...","LINESTRING (-122.438397 37.750226, -122.43832 ..."


In [68]:
%time blockfaces.to_file('../data/processed/blockfaces.geojson', driver='GeoJSON')

In [66]:
%time street_segments.to_file('../data/processed/street_segments.geojson', driver='GeoJSON')

CPU times: user 25.5 s, sys: 213 ms, total: 25.8 s
Wall time: 26.7 s
